## Grupo Bimbo Inventory Demand
https://www.kaggle.com/c/grupo-bimbo-inventory-demand

E-commerce website: http://blumonpay.net/bimbo/

In [1]:
dir = "C:/github/kaggle-fiddling/bimbo/"

In [2]:
import numpy as np 
import pandas as pd 
import bcolz

#import ipython_memory_usage as imu
#imu.start_watching_memory()

bcolz.blosc_set_nthreads(8)

8

In [3]:
# read from CSV ; comment-in next lines when done
train = pd.read_csv(dir + 'train.csv', dtype={
    'Semana': np.int8,
    'Agencia_ID': np.int32,
    'Canal_ID': np.int16,
    'Ruta_SAK': np.int16,
    'Cliente_ID': np.int64,
    'Producto_ID': np.int32,
    'Venta_uni_hoy': np.int16,
    'Venta_hoy': np.float16,
    'Dev_uni_proxima': np.int32,
    'Dev_proxima': np.float16,
    'Demanda_uni_equil': np.int16
})

In [4]:
# generate bcolz file; comment-in next lines when done

zTrain = bcolz.ctable.fromdataframe(train)
zTrain.copy(rootdir=dir+"train.bcolz", mode='w')

ctable((74180464,), [('Semana', 'i1'), ('Agencia_ID', '<i4'), ('Canal_ID', '<i2'), ('Ruta_SAK', '<i2'), ('Cliente_ID', '<i8'), ('Producto_ID', '<i4'), ('Venta_uni_hoy', '<i2'), ('Venta_hoy', '<f2'), ('Dev_uni_proxima', '<i4'), ('Dev_proxima', '<f2'), ('Demanda_uni_equil', '<i2')])
  nbytes: 2.28 GB; cbytes: 586.13 MB; ratio: 3.98
  cparams := cparams(clevel=5, shuffle=1, cname='blosclz')
  rootdir := 'C:/github/kaggle-fiddling/bimbo/train.bcolz'
[(3, 1110, 7, 3301, 15766, 1212, 3, 25.140625, 0, 0.0, 3)
 (3, 1110, 7, 3301, 15766, 1216, 4, 33.53125, 0, 0.0, 4)
 (3, 1110, 7, 3301, 15766, 1238, 4, 39.3125, 0, 0.0, 4) ...,
 (9, 25759, 1, 5517, 4528866, 45112, 4, 51.0, 0, 0.0, 4)
 (9, 25759, 1, 5517, 4547943, 40217, 3, 26.9375, 3, 26.9375, 0)
 (9, 25759, 1, 5517, 4708097, 43159, 1, 10.3984375, 0, 0.0, 1)]

In [ ]:
# once bcolz file has been generated, load it in the following way:

trainBZ = bcolz.ctable(rootdir = dir+"train.bcolz")
train = trainBZ.todataframe()

In [5]:
train[['Semana','Demanda_uni_equil']].groupby('Semana').sum()

,Demanda_uni_equil
Semana,
3,77664309.0
4,79618866.0
5,77610637.0
6,73851129.0
7,76597014.0
8,75525105.0
9,75054450.0


<img src="demanda-semana.png" alt="Demand / week" width="240" height="391">

Observations:
- wk 1 = January 1st ?
- no clear trend
- drop in wk 6: holidays ? 
- Mexico is a US-oriented touristic country. demand peek connected to US tourists ? to check this hypothesis, analyze geographical distribution using town_states.csv 

## Variabilily across regions

In [6]:
town = pd.read_csv(dir+'town_state.csv')

In [10]:
train_geo = pd.merge(left=train, right=town, how='left', on='Agencia_ID')

In [14]:
groups=train_geo[['Semana','State', 'Demanda_uni_equil']].groupby(['State', 'Semana']).sum()
pd.set_option('display.max_rows', 5000)
groups

Demanda_uni_equil
State                 Semana                   
AGUASCALIENTES        3               1349934.0
                      4               1388511.0
                      5               1376604.0
                      6               1301798.0
                      7               1345077.0
                      8               1310407.0
                      9               1250535.0
BAJA CALIFORNIA NORTE 3               2311226.0
                      4               2378229.0
                      5               2287456.0
                      6               2082907.0
                      7               2226876.0
                      8               2072414.0
                      9               2223824.0
BAJA CALIFORNIA SUR   3                493808.0
                      4                515434.0
                      5                503024.0
                      6                525415.0
                      7                518227.0
                      8                496840.0
                      9                482144.0
CAMPECHE              3                641795.0
                      4                641079.0
                      5                601333.0
                      6                581191.0
                      7                610651.0
                      8                602942.0
                      9                575022.0
CHIAPAS               3               1156111.0
                      4               1143037.0
                      5               1042349.0
                      6               1039218.0
                      7               1079235.0
                      8               1060765.0
                      9               1049972.0
CHIHUAHUA             3               2166785.0
                      4               2286330.0
                      5               2288468.0
                      6               2232527.0
                      7               2288421.0
                      8               2182913.0
                      9               2171726.0
COAHUILA              3               1994428.0
                      4               2073983.0
                      5               2065241.0
                      6               1969140.0
                      7               1944317.0
                      8               1887723.0
                      9               1910698.0
COLIMA                3                708864.0
                      4                701360.0
                      5                685796.0
                      6                685332.0
                      7                734965.0
                      8                711381.0
                      9                696382.0
DURANGO               3                911061.0
                      4                958855.0
                      5                998401.0
                      6                995255.0
                      7                971730.0
                      8                956087.0
                      9                936185.0
ESTADO DE MÉXICO      3              10552261.0
                      4              10489671.0
                      5              10424028.0
                      6               9723778.0
                      7              10040746.0
                      8              10310433.0
                      9              10144764.0
GUANAJUATO            3               4022991.0
                      4               4220050.0
                      5               4149190.0
                      6               3748845.0
                      7               3972672.0
                      8               3979894.0
                      9               3790971.0
GUERRERO              3               1625520.0
                      4               1626772.0
                      5               1605490.0
                      6               1621027.0
                      7               1542531.

### There is a strong variability across regions indeed